In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import RFECV
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from feature_engine.encoding import OneHotEncoder
from feature_engine.encoding import OrdinalEncoder

import pyodbc
from sqlalchemy import create_engine

In [2]:
connection_string = 'postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{database}'.format(
    username='admin1@np-ado-asg2-jairzhou',
    password='jair_zhou1',
    hostname='np-ado-asg2-jairzhou.postgres.database.azure.com',
    port='5432',
    database='postgres'
)
engine = create_engine(connection_string)

ecom1 = pd.read_sql_query("SELECT * FROM ecom1", engine)
ecom2 = pd.read_sql_query("SELECT * FROM ecom2", engine)
ecom3 = pd.read_sql_query("SELECT * FROM ecom3", engine)

engine.dispose()

In [3]:
ecom1.head()

,churn,user_id,push_status,app_sessions,location_code,account_length,add_to_wishlist,avg_order_value,app_transactions,desktop_sessions,...,credit_card_info_save,customer_service_calls,add_to_cart_per_session,total_product_detail_views,discount_rate_per_visited_products,product_detail_view_per_app_session,_airbyte_ab_id,_airbyte_emitted_at,_airbyte_normalized_at,_airbyte_ecom1_hashid
0,0,3824657,yes,45,415,128,25,"244,7",3,265,...,no,1,"2,7",110,"11,01",10,31eac9e4-4bc8-4b15-87a2-9140c60e1083,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:04.980496+00:00,832ceb1e8f1ca2b6edd49c318e8a5c9d
1,0,3717191,yes,27,415,107,26,"254,4",3,162,...,no,1,"3,7",123,"11,45","13,7",2b62d5f9-b0bc-4372-b89b-a2d63c5a17ac,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:04.980496+00:00,af32eb0b0653718c5e2a5d8caa79f0dd
2,0,3581921,no,41,415,137,0,"162,6",5,243,...,no,0,"3,29",114,"7,32","12,2",cf4aa7b2-efd7-41f3-a76a-2340e3636cc5,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:04.980496+00:00,b79a13c0125792f468f14ec20b213c22
3,0,3759999,no,51,408,84,0,"196,9",7,299,...,yes,2,"1,78",71,"8,86","6,6",2571c5a1-748e-4aa6-aaee-d573ad659ee7,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:04.980496+00:00,603e526511668b4e1d06cc5d6a3d9514
4,0,3306626,no,28,415,75,0,"186,9",3,167,...,yes,3,"2,73",113,"8,41","10,1",41df6016-602b-492a-a709-3afa8353575d,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:04.980496+00:00,cde31ab428f05f0c8d75e596481a507d


In [4]:
ecom2.head()

,churn,gender,tenure,citytier,complain,couponused,customerid,ordercount,maritalstatus,cashbackamount,...,daysincelastorder,satisfactionscore,preferredlogindevice,preferredpaymentmode,numberofdeviceregistered,orderamounthikefromlastyear,_airbyte_ab_id,_airbyte_emitted_at,_airbyte_normalized_at,_airbyte_ecom2_hashid
0,1,Female,4.0,3,1,1.0,50001,1.0,Single,160,...,5.0,2,Mobile Phone,Debit Card,3,11.0,c680ed4a-8646-4545-a8c5-dcd1540ed287,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:04.980496+00:00,4a06bcfb0b89049e01fd00c6748c8c35
1,1,Male,NaN,1,1,0.0,50002,1.0,Single,121,...,0.0,3,Phone,UPI,4,15.0,fadc2748-106b-44a1-9fd4-028cad603ae1,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:04.980496+00:00,a706a93b02610572e8321640520bee72
2,1,Male,NaN,1,1,0.0,50003,1.0,Single,120,...,3.0,3,Phone,Debit Card,4,14.0,1e27126b-e093-4c34-9c09-9593ca4f3fdb,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:04.980496+00:00,5faa716f9cdc0dbee232d4326f70af07
3,1,Male,0.0,3,0,0.0,50004,1.0,Single,134,...,3.0,5,Phone,Debit Card,4,23.0,9a0c1057-a142-432b-92e6-ad6bd4c53db6,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:04.980496+00:00,bbfb3045a90299a337a0b70763b62ec5
4,1,Male,0.0,1,0,1.0,50005,1.0,Single,130,...,3.0,5,Phone,CC,3,11.0,b51131fd-2679-4065-9016-d35ff3beb280,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:04.980496+00:00,a9aa1b6d61fb6211db645d61cb308df6


In [5]:
ecom3.head()

,int_n,ses_n,tran_n,int_n_r,rev_sum,ses_n_r,ses_rec,tran_n_r,user_rec,rev_sum_r,...,time_to_int,target_class,time_to_tran,int_cat_n_avg,int_itm_n_avg,major_spend_r,_airbyte_ab_id,_airbyte_emitted_at,_airbyte_normalized_at,_airbyte_ecom3_hashid
0,3.0,2.0,0.0,1.500000,0.0,0.051282,23.0,0.0,39.0,0.0,...,1.911111,1,-1.0,1.0,1.5,0.0,63916787-8158-4329-a27f-1e4ca90883cc,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:05.121083+00:00,920e7f27b0596a4cdae7e4cf0ef4f808
1,2.0,2.0,0.0,1.000000,0.0,0.023529,23.0,0.0,85.0,0.0,...,0.000000,1,-1.0,1.0,1.0,0.0,69bff9ce-6c10-431e-a497-e8fcda3fe098,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:05.121083+00:00,c6344f033b171f18e0d0b360c344e098
2,13.0,11.0,0.0,1.181818,0.0,0.523810,14.0,0.0,21.0,0.0,...,0.934615,1,-1.0,1.0,1.0,0.0,441127fb-9a99-4787-a87e-cba9082a048f,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:05.121083+00:00,ee35d00fda57eea9d7316b5f022a1c8e
3,4.0,2.0,0.0,2.000000,0.0,0.083333,11.0,0.0,24.0,0.0,...,0.254167,1,-1.0,1.5,1.5,0.0,ccde7bbe-0e4d-4dab-b961-1347b772453c,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:05.121083+00:00,746b6de692b27d904420bcd44fac2f8d
4,2.0,2.0,0.0,1.000000,0.0,0.046512,16.0,0.0,43.0,0.0,...,0.000000,1,-1.0,1.0,1.0,0.0,23c78980-55d7-4691-a885-5e75159f228d,2023-01-24 12:35:55.026000+00:00,2023-01-24 12:37:05.121083+00:00,0c951dcbaf12b4f950534daa595668fa


In [6]:
df = ecom2

In [7]:
# drop irrelevant data
df.drop(list(df.filter(regex = '_airbyte_')), axis = 1, inplace = True)
df.head()

,churn,gender,tenure,citytier,complain,couponused,customerid,ordercount,maritalstatus,cashbackamount,hourspendonapp,numberofaddress,warehousetohome,preferedordercat,daysincelastorder,satisfactionscore,preferredlogindevice,preferredpaymentmode,numberofdeviceregistered,orderamounthikefromlastyear
0,1,Female,4.0,3,1,1.0,50001,1.0,Single,160,3.0,9,6.0,Laptop & Accessory,5.0,2,Mobile Phone,Debit Card,3,11.0
1,1,Male,NaN,1,1,0.0,50002,1.0,Single,121,3.0,7,8.0,Mobile,0.0,3,Phone,UPI,4,15.0
2,1,Male,NaN,1,1,0.0,50003,1.0,Single,120,2.0,6,30.0,Mobile,3.0,3,Phone,Debit Card,4,14.0
3,1,Male,0.0,3,0,0.0,50004,1.0,Single,134,2.0,8,15.0,Laptop & Accessory,3.0,5,Phone,Debit Card,4,23.0
4,1,Male,0.0,1,0,1.0,50005,1.0,Single,130,NaN,3,12.0,Mobile,3.0,5,Phone,CC,3,11.0


In [8]:
df.isnull().sum()

churn                            0
gender                           0
tenure                         264
citytier                         0
complain                         0
couponused                     256
customerid                       0
ordercount                     258
maritalstatus                    0
cashbackamount                   0
hourspendonapp                 255
numberofaddress                  0
warehousetohome                251
preferedordercat                 0
daysincelastorder              307
satisfactionscore                0
preferredlogindevice             0
preferredpaymentmode             0
numberofdeviceregistered         0
orderamounthikefromlastyear    265
dtype: int64

In [9]:
df['preferredpaymentmode'] = df['preferredpaymentmode'].str.replace('Cash on Delivery','COD')
df['preferredpaymentmode'] = df['preferredpaymentmode'].str.replace('Credit Card','CC')

In [10]:
df = df.replace(' ', '_', regex=True)
df = df.replace('&', 'And', regex=True)

In [11]:
X = df.drop(columns=['customerid','churn'])
y = df['churn']
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, random_state=42)

In [12]:
def label_encoding(x_train,
                   x_test):
  ordinal_encoder = OrdinalEncoder(encoding_method='arbitrary',
variables='gender')
  ordinal_encoder.fit(x_train)
  x_train = ordinal_encoder.transform(x_train)
  x_test = ordinal_encoder.transform(x_test)
  return x_train, x_test

X_train, X_test = label_encoding(X_train,X_test)

In [13]:
def label_ohe(x_train, 
              x_test):
  cat_features = ['preferredlogindevice', 'preferredpaymentmode', 'preferedordercat', 'maritalstatus']
  ohe_encoder = OneHotEncoder(variables=cat_features)
  ohe_encoder.fit(x_train)
  x_train = ohe_encoder.transform(x_train)
  x_test = ohe_encoder.transform(x_test)

  return x_train, x_test

X_train , X_test=label_ohe(X_train, X_test)

In [14]:
cat_features = ['tenure','warehousetohome','orderamounthikefromlastyear','couponused','ordercount','daysincelastorder']
cont_features = ['hourspendonapp']

In [15]:
def imputation (x_train, 
                x_test,
                cont_features,
                cat_features,
                cont_strategy = 'median',
                cat_strategy = 'most_frequent'
                ):

#initializa object imputer
  imputerCat = SimpleImputer(missing_values=np.nan, strategy= cat_strategy)
  imputerCont = SimpleImputer(missing_values=np.nan, strategy= cont_strategy)

#fit to data train
  imputerCont.fit(x_train[cont_features])
  imputerCat.fit(x_train[cat_features])

#transfrom to data train
  x_train[cont_features] = imputerCont.transform(x_train[cont_features])
  x_train[cat_features] = imputerCat.transform(x_train[cat_features])
  x_test[cont_features] = imputerCont.transform(x_test[cont_features])
  x_test[cat_features] = imputerCat.transform(x_test[cat_features])

  return x_train, x_test

X_train , X_test = imputation(X_train, X_test,cont_features, cat_features)

In [16]:
# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing dataabs
X_test_norm = norm.transform(X_test)

In [17]:
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

print(type(rfc))
scores = cross_val_score(rfc,X_train,y_train,cv=5, scoring='roc_auc')
print ("CV score :",scores.mean())
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
print(classification_report(y_test, pred))

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
CV score : 0.9800240643173723
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       924
           1       0.97      0.83      0.89       202

    accuracy                           0.96      1126
   macro avg       0.96      0.91      0.94      1126
weighted avg       0.96      0.96      0.96      1126



In [18]:
gs_rfc = GridSearchCV(RandomForestClassifier(random_state=42), {'max_depth': [2, 5, 8, 10, 12, 15, 20, 30, 50],
                                                 'n_estimators': [5, 10, 20, 30, 50, 100,200]},
                      verbose=1, cv=5) # make sure cv condition same as actual fitting for training set

gs_rfc.fit(X_train, y_train)

# select the best estimator
print('Best score:', gs_rfc.best_score_)
print('Best parameters:', gs_rfc.best_params_)

Fitting 5 folds for each of 63 candidates, totalling 315 fits
Best score: 0.9573718091009991
Best parameters: {'max_depth': 30, 'n_estimators': 200}


In [19]:
rfc = RandomForestClassifier(random_state=42,max_depth=30, n_estimators=200)

print(type(rfc))
scores = cross_val_score(rfc,X_train,y_train,cv=5, scoring='roc_auc')
print ("CV score :",scores.mean())
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
print(classification_report(y_test, pred))

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
CV score : 0.9813447669913072
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       924
           1       0.96      0.85      0.90       202

    accuracy                           0.97      1126
   macro avg       0.96      0.92      0.94      1126
weighted avg       0.97      0.97      0.96      1126



In [20]:
col_labels = X_train.columns
ss_importance = rfc.feature_importances_
df_ss_importance = pd.DataFrame(col_labels, columns=["feature"])
df_ss_importance["importance"] = ss_importance
df_ss_importance.sort_values("importance",ascending = False)

,feature,importance
1,tenure,0.221725
6,cashbackamount,0.079616
9,warehousetohome,0.067764
8,numberofaddress,0.061175
10,daysincelastorder,0.059650
13,orderamounthikefromlastyear,0.057781
3,complain,0.056825
11,satisfactionscore,0.044870
12,numberofdeviceregistered,0.038599
5,ordercount,0.032694
